In [2]:
with open('simple.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]


In [3]:
for idx, line in enumerate(lines):
    if line == '':
        workflows = lines[:idx]
        parts = lines[idx+1:]


In [4]:
workflows_dict = {}
for workflow in workflows: 
    workflow_index = workflow.split('{')[0]
    workflow_procedure = workflow.split('{')[1].split('}')[0]
    workflow_procedure = workflow_procedure.split(',')
    workflows_dict[workflow_index] = workflow_procedure

In [5]:
def go_through_workflows(workflow_idx, xmas_dict):
    workflow = workflows_dict[workflow_idx]
    for step in workflow:
        if ':' in step:
            condition = step.split(':')[0]
            target = step.split(':')[1]
            if '<' in condition: 
                variable = condition.split('<')[0]
                value = condition.split('<')[1]
                if xmas_dict[variable] < int(value):
                    return target
            elif '>' in condition:
                variable = condition.split('>')[0]
                value = condition.split('>')[1]
                if xmas_dict[variable] > int(value):
                    return target

        else:
            return step


In [7]:
sum_total = 0
for part in parts: 
    part = part[1:-1]
    xmas = part.split(',')
    xmas_dict = {}
    for letter in xmas:
        xmas_dict[letter.split('=')[0]] = int(letter.split('=')[1])
    
    a = go_through_workflows('in', xmas_dict)
    while a != 'A' and a != 'R':
        a = go_through_workflows(a, xmas_dict)
    
    if a == 'A':
        sum_ = 0
        for key in xmas_dict.keys():
            sum_ += xmas_dict[key]
        sum_total += sum_

In [8]:
sum_total

19114

Part 2 

In [10]:
workflows

['px{a<2006:qkq,m>2090:A,rfg}',
 'pv{a>1716:R,A}',
 'lnx{m>1548:A,A}',
 'rfg{s<537:gd,x>2440:R,A}',
 'qs{s>3448:A,lnx}',
 'qkq{x<1416:A,crn}',
 'crn{x>2662:A,R}',
 'in{s<1351:px,qqz}',
 'qqz{s>2770:qs,m<1801:hdj,R}',
 'gd{a>3333:R,R}',
 'hdj{m>838:A,pv}']

In [26]:
paths = []
def depth_first(index, path):
    if index == 'A':
        paths.append(path)
        return path
    if index == 'R':
        return None
    else:
        conditions = ''
        for step_idx, step in enumerate(workflows_dict[index]):
            # print('Workflow', index, 'Step', step_idx, '(Path', path, ')' )
            if ':' in step:
                condition = step.split(':')[0]
                target = step.split(':')[1]
                depth_first(target, path + conditions+','+condition) #branch 1
                if '<' in condition:
                    conditions += ','+condition.replace('<', '>')
                elif '>' in condition:
                    conditions += ','+condition.replace('>', '<')

            else:
                depth_first(step,  path + conditions)
                

In [27]:
depth_first('in' , '')

In [28]:
#167409079868000

In [29]:
paths

[',s<1351,a<2006,x<1416',
 ',s<1351,a<2006,x>1416,x>2662',
 ',s<1351,a>2006,m>2090',
 ',s<1351,a>2006,m<2090,s>537,x<2440',
 ',s>1351,s>2770,s>3448',
 ',s>1351,s>2770,s<3448,m>1548',
 ',s>1351,s>2770,s<3448,m<1548',
 ',s>1351,s<2770,m<1801,m>838',
 ',s>1351,s<2770,m<1801,m<838,a<1716']

In [57]:
paths_dicts = {}
for path_idx, path in enumerate(paths):
    variables_max = {'x': 4000, 'm': 4000, 'a': 4000, 's': 4000}
    variables_min = {'x': 1, 'm': 1, 'a': 1, 's': 1}
    conditions = path.split(',')
    for condition in conditions:
        if '<' in condition:
            variable = condition.split('<')[0]
            value = int(condition.split('<')[1])
            variables_max[variable] = value -1
        elif '>' in condition:
            variable = condition.split('>')[0]
            value = int(condition.split('>')[1])
            variables_min[variable] = value + 1
    variables_diff = {
        'x': variables_max['x'] - variables_min['x'] + 1,
        'm': variables_max['m'] - variables_min['m'] + 1,
        'a': variables_max['a'] - variables_min['a'] + 1,
        's': variables_max['s'] - variables_min['s'] + 1
    }
    paths_dicts[path_idx] = variables_diff

In [58]:
paths_dicts

{0: {'x': 1415, 'm': 4000, 'a': 2005, 's': 1350},
 1: {'x': 1338, 'm': 4000, 'a': 2005, 's': 1350},
 2: {'x': 4000, 'm': 1910, 'a': 1994, 's': 1350},
 3: {'x': 2439, 'm': 2089, 'a': 1994, 's': 813},
 4: {'x': 4000, 'm': 4000, 'a': 4000, 's': 552},
 5: {'x': 4000, 'm': 2452, 'a': 4000, 's': 677},
 6: {'x': 4000, 'm': 1547, 'a': 4000, 's': 677},
 7: {'x': 4000, 'm': 962, 'a': 4000, 's': 1418},
 8: {'x': 4000, 'm': 837, 'a': 1715, 's': 1418}}

In [59]:
#paths_dicts
sum_= 0
for key, value in paths_dicts.items():
    sum_ += value['x']*value['m']*value['a']*value['s']

sum_


167245503449662

In [60]:
167409079868000

167409079868000

In [152]:
letters_min = {}
letters_max = {}
for path in paths:
    splits = path.split(',')
    for split in splits:
        if '<' in split:
            variable = split.split('<')[0]
            value = split.split('<')[1]
            if variable not in letters_max.keys():
                letters_max[variable] = int(value)
            else:
                letters_max[variable] = min(letters_max[variable], int(value))
        elif '>' in split:
            variable = split.split('>')[0]
            value = split.split('>')[1]
            value = 4000 - value
            
            letters_min[variable] = int(value)
            


In [153]:
letters_max

{'s': 1351, 'a': 1716, 'x': 1416, 'm': 838}

In [154]:
letters_min

{'x': 2662, 'a': 2006, 'm': 2090, 's': 3448}

In [85]:
results = go_through_workflows('in')
target = None
while target != 'A' and target != 'R':
    for results in results:
        if results[1] == None:
    target = results[0]
    results = go_through_workflows(target)

In [ ]:
    for step in workflow:
        if ':' in step:
            condition = step.split(':')[0]
            target = step.split(':')[1]
            if '<' in condition: 
                variable = condition.split('<')[0]
                value = condition.split('<')[1]
                return target, variable, int(value)
            elif '>' in condition:
                variable = condition.split('>')[0]
                value = condition.split('>')[1]
                return target, variable, 4000 - int(value)
            
        else:
            return step, None, None


In [78]:
target, variable, value = go_through_workflows('in')
while target != 'A':
    print(target, variable, value)
    target, variable, value = go_through_workflows(target)
    

px s 1351
qkq a 2006


In [75]:
sum_total = 0
for part in parts: 
    print(part)
    part = part[1:-1]
    xmas = part.split(',')
    xmas_dict = {}
    for letter in xmas:
        xmas_dict[letter.split('=')[0]] = int(letter.split('=')[1])
    
    upper_limits = {}
    target, variable, value = go_through_workflows('in', xmas_dict)
    upper_limits[variable] = value
    while target != 'A' and target != 'R':
        target, variable, value = go_through_workflows(target, xmas_dict)
        upper_limits[variable] = value
    
    print(upper_limits)
    if target == 'A':
        possibilities = 1
        for key in upper_limits.keys():
            if key is not None:
                possibilities *= upper_limits[key]
        print(possibilities)
        sum_total += possibilities
    break

{x=787,m=2655,a=1222,s=2876}
{None: None, 's': 1230, 'm': 2452}
3015960


In [74]:
sum_total

3019112